# 0. Data

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [41]:
'split data'
'first 21 columns are input variables and last 7 columns are output variables'

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.iloc[:,:21], train_data.iloc[:,-7:], test_size=0.2)
X_test, Y_test = test_data.iloc[:,:21], test_data.iloc[:,-7:]

print('train set: ', X_train.shape, Y_train.shape)
print('valid set: ', X_valid.shape, Y_valid.shape)
print('test set: ', X_test.shape, Y_test.shape)

train set:  (31587, 21) (31587, 7)
valid set:  (7897, 21) (7897, 7)
test set:  (5000, 21) (5000, 7)


# 1. Nonlinear GP regression
- Gaussian kernel

In [44]:
class GaussianKernel:
    """
    isotropic Gaussian kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix C (N*M)
    """
    def __init__(self, sigma_k=1):
        self.sigma_k = sigma_k # kernel variance (Hyperparameter !)

    def __call__(self, X1, X2):
        C = np.exp(-(np.sum(X1**2, axis=1).values.reshape(-1, 1) +
                        np.sum(X2**2, axis=1).values.reshape(1, -1) - 2*X1@X2.T) / pow(self.sigma_k, 2) )
        return C

In [ ]:
class GaussianProcessRegression:
    def __init__(self, X_train, Y_train, K=GaussianKernel(), sigma_n=1):
        self.X_train = X_train
        self.Y_train = Y_train
        self.K = K
        self.sigma_n = sigma_n  # noise variance (Hyperparameter !)
        self.predictive_distribution = None
        self.C_train = K(X_train, X_train)

    def predict(self, X_test):
        D = self.Y_train.shape[1]
        c_matrix = np.identity(D) # coregionalisation matrix (used for vector-valued model)
        K_Xt_X = self.K(X_test, self.X_train)
        K_X_X = self.K(self.X_train, self.X_train)
        Sigma = self.sigma_n * np.identity(D)
        y_concat = self.Y_train



In [55]:
Y_train

,21,22,23,24,25,26,27
36152,39.037277,-44.735386,22.403183,52.728859,-1.618516,-2.946491,8.427116
5000,-15.615760,-60.307045,40.478531,28.765852,2.852310,-1.158578,7.125257
25415,13.456683,-20.796692,10.498929,27.005915,-0.970999,0.251025,3.572284
21629,1.902965,-13.864461,2.670248,15.111175,-0.849624,-0.579289,2.433015
34067,32.757721,-55.250778,21.193287,46.750130,-2.362372,-2.083188,8.284686
...,...,...,...,...,...,...,...
24496,6.116674,-31.806705,15.839424,35.016659,-0.182062,-2.471634,5.889440
30228,-8.291492,-26.913366,4.066059,7.221807,-1.395811,-1.486842,0.077239
12245,13.456683,-28.544479,17.174549,36.169720,0.546187,-3.533664,6.159775
4504,-2.920606,-16.295956,6.179588,18.568508,-0.940735,-1.660629,3.186091


In [63]:
np.array(Y_train).reshape(-1, 1)

array([[ 39.037277],
       [-44.735386],
       [ 22.403183],
       ...,
       [ -1.395811],
       [ -0.946172],
       [  0.54067 ]])

# 2.Linear GP regression 
- Standard dot product kernel
    - k(a,b) = a^\top b